In [ ]:
# python3
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [ ]:
from ActualLatLong import *

In [ ]:
path = "corr.csv"
df_corr = pd.read_csv(path)
del df_corr['Unnamed: 0']

""" add latlong back """
if 'lat' not in df_corr.columns:
    df_corr['lat'] = (df_corr['latlong'] // 1000.).astype(int)
    df_corr['long'] = (df_corr['latlong'] % 1000.).astype(int)

In [ ]:
""" get new lat_long """
new_lat_long = []

for y in range(0, -480, -1):
    for x in range(480):
        distance, angle = find_distance_angle(y, x)
        lat, long = displace(ORIGIN_LAT, ORIGIN_LONG, angle, distance)
        new_lat_long.append([-y, x, lat, long])

In [ ]:
ll_arr = np.array(new_lat_long)
test = pd.DataFrame(ll_arr, columns=["lat","long" ,"map_lat", "map_long"])
test['lat'] = test['lat'].astype(int)
test['long'] = test['long'].astype(int)
# test.to_csv("map_latlong.csv",index=False)

## Folium Playground

In [ ]:
import folium
from folium import plugins
from folium.plugins import HeatMap
import pandas as pd
import numpy as np

In [ ]:
"""get centre of map"""
test = pd.read_csv("processed_data/map_latlong.csv")
mid = test.loc[test['lat'] == 239].loc[test['long'] == 239]
mid_lat = mid['map_lat'].values[0]
mid_long = mid['map_long'].values[0]
m = folium.Map([mid_lat, mid_long], zoom_start=10)

In [ ]:
cols = df_corr.columns.values[21]
df_corr['map_lat'] = test['map_lat']
df_corr['map_long'] = test['map_long']
temp = df_corr[['map_lat', 'map_long', cols]].copy()
temp = temp.fillna(0, inplace=False)
temp.isnull().sum()

In [ ]:
"""filter by median"""
temp.max(axis=0)
med = temp.median(axis=0).corr_station_20
temp_filtered = temp.loc[temp['corr_station_20'] > med]

In [ ]:
"""filter by percentile"""
min = temp.quantile(q=0.999, axis=0).corr_station_20
temp_filtered = temp.loc[temp[cols] > min]
temp_filtered[cols] = temp_filtered[cols]

In [26]:
m = folium.Map([mid_lat, mid_long], zoom_start=10)
max_amount = float(temp_filtered[cols].max())

data = np.array(temp_filtered).tolist()
HeatMap(data,
        min_opacity=0.9,
        max_val=max_amount,
        radius=7, blur=14,
        max_zoom=3
       ).add_to(m)
m

In [ ]:
"""generate html visualization on map"""
for cols in df_corr.columns.values[1:51]:
    print(cols)
    temp = df_corr[['map_lat', 'map_long', cols]].copy()
    temp = temp.fillna(0, inplace=False)
    temp.isnull().sum()

    min_amount = temp.quantile(q=0.999, axis=0)[cols]
    temp_filtered = temp.loc[temp[cols] > min_amount]
    temp_filtered[cols] = temp_filtered[cols]*1000
    max_amount = float(temp_filtered[cols].max())
    
    m = folium.Map([mid_lat, mid_long], zoom_start=10)
    data = np.array(temp_filtered).tolist()
    HeatMap(data,
            min_opacity=0.9,
            max_val=max_amount,
            radius=7, blur=14,
            max_zoom=3
           ).add_to(m)
    m.save('heatmaps/heatmap_'+cols+'.html')